In [1]:
import tensorflow as tf

def create_model(input_shape=(40,)):
  # Create a 3-hidden-layer fully-connected NN
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.InputLayer(input_shape=input_shape))
  model.add(tf.keras.layers.Dense(30, kernel_initializer='glorot_uniform', use_bias=False, activation=None, name='dense'))
  model.add(tf.keras.layers.BatchNormalization(momentum=0.99, epsilon=1e-4, name='batch_normalization'))
  model.add(tf.keras.layers.Activation('tanh', name='activation'))
  model.add(tf.keras.layers.Dense(20, kernel_initializer='glorot_uniform', use_bias=False, activation=None, name='dense_1'))
  model.add(tf.keras.layers.BatchNormalization(momentum=0.99, epsilon=1e-4, name='batch_normalization_1'))
  model.add(tf.keras.layers.Activation('tanh', name='activation_1'))
  model.add(tf.keras.layers.Dense(10, kernel_initializer='glorot_uniform', use_bias=False, activation=None, name='dense_2'))
  model.add(tf.keras.layers.BatchNormalization(momentum=0.99, epsilon=1e-4, name='batch_normalization_2'))
  model.add(tf.keras.layers.Activation('tanh', name='activation_2'))
  model.add(tf.keras.layers.Dense(1, kernel_initializer='glorot_uniform', use_bias=False, activation=None, name='dense_3'))
  model.compile(loss='mse', optimizer='adam')
  model.summary()
  return model

def save_model(model, name=None):
  # Save as model.h5, model_weights.h5, and model.json
  if name is None:
    name = model.name
  model.save(name + '.h5')
  model.save_weights(name + '_weights.h5')
  with open(name + '.json', 'w') as outfile:
    outfile.write(model.to_json())
  return

if __name__ == '__main__':
  model = create_model()
  save_model(model, name='model')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                1200      
                                                                 
 batch_normalization (BatchN  (None, 30)               120       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 30)                0         
                                                                 
 dense_1 (Dense)             (None, 20)                600       
                                                                 
 batch_normalization_1 (Batc  (None, 20)               80        
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 20)                0

In [2]:
model = create_model()
save_model(model, name='model')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                1200      
                                                                 
 batch_normalization (BatchN  (None, 30)               120       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 30)                0         
                                                                 
 dense_1 (Dense)             (None, 20)                600       
                                                                 
 batch_normalization_1 (Batc  (None, 20)               80        
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 20)               

In [3]:
import hls4ml
config = hls4ml.utils.config_from_keras_model(model, granularity='name')

Interpreting Sequential
Topology:
Layer name: input_2, layer type: Input
Layer name: dense, layer type: Dense
  -> Activation (linear), layer name: dense
Layer name: batch_normalization, layer type: BatchNormalization
Layer name: activation, layer type: Activation
Layer name: dense_1, layer type: Dense
  -> Activation (linear), layer name: dense_1
Layer name: batch_normalization_1, layer type: BatchNormalization
Layer name: activation_1, layer type: Activation
Layer name: dense_2, layer type: Dense
  -> Activation (linear), layer name: dense_2
Layer name: batch_normalization_2, layer type: BatchNormalization
Layer name: activation_2, layer type: Activation
Layer name: dense_3, layer type: Dense
  -> Activation (linear), layer name: dense_3


/home/chmoser/miniconda3/envs/tf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
hls_model = hls4ml.converters.convert_from_keras_model(model, hls_config=config, output_dir='test_prj')

Interpreting Sequential
Topology:
Layer name: input_2, layer type: InputLayer, input shapes: [[None, 40]], output shape: [None, 40]
Layer name: dense, layer type: Dense, input shapes: [[None, 40]], output shape: [None, 30]
Layer name: batch_normalization, layer type: BatchNormalization, input shapes: [[None, 30]], output shape: [None, 30]
Layer name: activation, layer type: Activation, input shapes: [[None, 30]], output shape: [None, 30]
Layer name: dense_1, layer type: Dense, input shapes: [[None, 30]], output shape: [None, 20]
Layer name: batch_normalization_1, layer type: BatchNormalization, input shapes: [[None, 20]], output shape: [None, 20]
Layer name: activation_1, layer type: Activation, input shapes: [[None, 20]], output shape: [None, 20]
Layer name: dense_2, layer type: Dense, input shapes: [[None, 20]], output shape: [None, 10]
Layer name: batch_normalization_2, layer type: BatchNormalization, input shapes: [[None, 10]], output shape: [None, 10]
Layer name: activation_2, lay

In [7]:
hls_model.write()

Writing HLS project
Done


In [1]:
from keras.layers import *
from qkeras import *

x = x_in = Input(shape)
x = QConv2D(18, (3, 3),
        kernel_quantizer="stochastic_ternary",
        bias_quantizer="ternary", name="first_conv2d")(x)
x = QActivation("quantized_relu(3)")(x)
x = QSeparableConv2D(32, (3, 3),
        depthwise_quantizer=quantized_bits(4, 0, 1),
        pointwise_quantizer=quantized_bits(3, 0, 1),
        bias_quantizer=quantized_bits(3),
        depthwise_activation=quantized_tanh(6, 2, 1))(x)
x = QActivation("quantized_relu(3)")(x)
x = Flatten()(x)
x = QDense(NB_CLASSES,
        kernel_quantizer=quantized_bits(3),
        bias_quantizer=quantized_bits(3))(x)
x = QActivation("quantized_bits(20, 5)")(x)
x = Activation("softmax")(x)

NameError: name 'shape' is not defined